In [11]:
import pandas as pd
import numpy as np
import glob
import sqlite3
import pymysql


path = 'data/large' # use your path
all_files = glob.glob(path + "/POWER*.csv")

for filename in all_files:
    df_ap = pd.read_csv(filename, index_col=None, header=10)
    li.append(df_ap)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['DATE'] = frame['YEAR'].map(str) + '-' + pd.to_datetime(frame['DOY'], format='%j').dt.strftime('%m-%d')
frame.drop(['YEAR', 'DOY'], axis=1, inplace=True)
frame.columns = ['lat', 'lon', 'temp', 'date']

In [18]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Dementors1!',
                         db='ae2_3')


# create cursor
cursor=connection.cursor()

li = []
# creating column list for insertion
cols = "`,`".join([str(i) for i in frame.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in frame.iterrows():
    sql = "INSERT INTO `temps` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [3]:
import sqlite3


# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/switrs.sqlite")
q = """
SELECT latitude, longitude, collision_date, weather_1 FROM collisions
WHERE collision_date IS NOT NULL
AND longitude IS NOT NULL
AND latitude IS NOT NULL    
AND collision_date <= '2017-12-31'  -- 2018 is incomplete
AND collision_date >= '2010-01-01'
"""

df = pd.read_sql_query(q, con)
df.columns = ['latitude', 'longitude', 'collision_date', 'weather']
df.drop(['latitude', 'longitude', 'collision_date'], axis=1, inplace=True)

# # Connect to the database
# connection = pymysql.connect(host='localhost',
#                          user='root',
#                          password='Dementors1!',
#                          db='ae2_3')


# # create cursor
# cursor=connection.cursor()

# li = []
# # creating column list for insertion
# cols = "`,`".join([str(i) for i in df.columns.tolist()])

# # Insert DataFrame recrds one by one.
# for i,row in df.iterrows():
#     sql = "INSERT INTO `collisions` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     cursor.execute(sql, tuple(row))

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     connection.commit()

In [7]:
import pandas as pd
df_temp = pd.read_csv('/Users/dantesean/Documents/Projects/ae-2/Week 3/Assignment/MySQL/ae2_3_total.csv')

In [7]:
import pymysql

In [9]:
df_temp['weather'] = df['weather']
df_temp.dropna(inplace=True)

In [11]:
1553379 - 1549530

3849

In [5]:
collisions_grouped_1 = pd.DataFrame(pd.Series(df_temp.groupby('collision_date').mean()['temp']))
collisions_grouped_1['count'] = pd.Series(df_temp['collision_date'].value_counts().sort_index())
collisions_grouped_1.columns = ['temp', 'count']
collisions_grouped_1.to_csv('data/large/collisions_grouped_1_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)

# Conditional on Rain
df_rain = df_temp[df_temp['weather'] == 'raining']
collisions_grouped_2 = pd.DataFrame(pd.Series(df_rain.groupby('collision_date').mean()['temp']))
collisions_grouped_2['count'] = pd.Series(df_rain['collision_date'].value_counts().sort_index())
collisions_grouped_2.columns = ['temp', 'count']
collisions_grouped_2_final = collisions_grouped_2.reindex(pd.DataFrame(pd.Series(df_temp.groupby('collision_date').mean()['temp'])).index, fill_value=0)
collisions_grouped_2_final.loc[collisions_grouped_2_final['temp'] == 0, 'temp'] = collisions_grouped_1['temp']
collisions_grouped_2_final.to_csv('data/large/collisions_grouped_2_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)

# Conditional on No Rain
df_no_rain = df_temp[df_temp['weather'] != 'raining']
collisions_grouped_3 = pd.DataFrame(pd.Series(df_no_rain.groupby('collision_date').mean()['temp']))
collisions_grouped_3['count'] = pd.Series(df_no_rain['collision_date'].value_counts().sort_index())
collisions_grouped_3.columns = ['temp', 'count']
collisions_grouped_3_final = collisions_grouped_3.reindex(pd.DataFrame(pd.Series(df_temp.groupby('collision_date').mean()['temp'])).index, fill_value=0)
collisions_grouped_3_final.loc[collisions_grouped_3_final['temp'] == 0, 'temp'] = collisions_grouped_1['temp']
collisions_grouped_3_final.to_csv('data/large/collisions_grouped_3_no_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False,)

<ipython-input-5-3e0a7be35c12>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  collisions_grouped_1.to_csv('data/large/collisions_grouped_1_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)
<ipython-input-5-3e0a7be35c12>:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  collisions_grouped_2_final.to_csv('data/large/collisions_grouped_2_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)
<ipython-input-5-3e0a7be35c12>:22: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  collisions_grouped_3_final.to_csv('data/large/collisions_grouped_3_no_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False,)
